# Hardware Model 1 on QUT

### A Simple Introduction to Spiking Neural Networks with NeuronovaSim
First, we need to import common libraries and Neuronova-Sim (N-Wave)

In [6]:
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# Import necessary components from your library
from Neuronova_Sim.NeuronovaSim import NeuronovaSim

In [7]:
# We set our seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

### Loading the dataset

We use mini_QUT, an internal dataset composed of the first 100 recordings for each class.

In [8]:
class CochleaDataset(Dataset):
    """
    Dataset class for loading neural response data organized in class-specific folders.
    Data structure:
    root_folder/
        class1/
            file1.csv
            file2.csv
        class2/
            file3.csv
            file4.csv
    """

    def __init__(self, root_folder, split="train", val_split=0.2, random_state=42):
        assert split in [
            "train",
            "val",
            "test",
        ], f"Split {split} not recognized. Use 'train', 'val', or 'test'"
        self.files = []
        self.labels = []
        self.class_to_idx = {}
        self.split = split

        # Get all class folders
        class_folders = [
            d
            for d in os.listdir(root_folder)
            if os.path.isdir(os.path.join(root_folder, d))
        ]

        # Save classes number
        self.n_classes = len(class_folders)

        # Create class mapping
        self.class_to_idx = {
            cls_name: idx for idx, cls_name in enumerate(sorted(class_folders))
        }

        print(f"Found {self.n_classes} classes: {list(self.class_to_idx.keys())}")

        # Collect all files and labels
        all_files = []
        all_labels = []
        for class_name in class_folders:
            class_path = os.path.join(root_folder, class_name)
            class_files = [
                os.path.join(class_path, f)
                for f in os.listdir(class_path)
                if f.endswith(".csv") and not f.startswith(".")
            ]
            all_files.extend(class_files)
            all_labels.extend([self.class_to_idx[class_name]] * len(class_files))
            print(f"Class '{class_name}': {len(class_files)} samples")

        # Create a deterministic split
        generator = torch.Generator().manual_seed(random_state)
        indices = torch.randperm(len(all_files), generator=generator)

        # Calculate split sizes
        val_size = int(len(indices) * val_split)
        train_size = len(indices) - val_size

        # Select appropriate indices based on split
        if split == "train":
            split_indices = indices[:train_size]
        elif split == "val":
            split_indices = indices[train_size:]
        elif split == "test":
            split_indices = indices

        # Use the selected indices to populate the dataset
        self.files = [all_files[i] for i in split_indices]
        self.labels = [all_labels[i] for i in split_indices]

        print(f"Using '{split}' split with {len(self.files)} samples")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        label = self.labels[idx]
        label = torch.tensor(label)
        label_one_hot = torch.nn.functional.one_hot(label, num_classes=self.n_classes)

        # Load neural response data
        # Each row is a time bin, each column is a neuron
        data = np.loadtxt(file_path, delimiter=",", dtype=np.float32, encoding="latin1")

        # Reshape to (time_bins, n_neurons)
        # No need to add extra dimension since we already have multiple neurons
        data = data.reshape(data.shape[0], -1)  # Force 2D shape in case single neuron

        # Convert to tensor with shape (time_bins, n_neurons)
        data_tensor = torch.tensor(data, dtype=torch.float32)

        return data_tensor, label_one_hot, file_path


# Dataset disk position and batchsize
data_dir = "data/4bit_frontend/mini_TRAIN_VAL"
batch_size = 32

# Create the datasets
train_dataset = CochleaDataset(data_dir, split="train")
val_dataset = CochleaDataset(data_dir, split="val")

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True
)

Found 4 classes: ['CAFE-mini', 'CAR-mini', 'HOME-mini', 'STREET-mini']
Class 'CAFE-mini': 96 samples
Class 'HOME-mini': 96 samples
Class 'CAR-mini': 96 samples
Class 'STREET-mini': 96 samples
Using 'train' split with 308 samples
Found 4 classes: ['CAFE-mini', 'CAR-mini', 'HOME-mini', 'STREET-mini']
Class 'CAFE-mini': 96 samples
Class 'HOME-mini': 96 samples
Class 'CAR-mini': 96 samples
Class 'STREET-mini': 96 samples
Using 'val' split with 76 samples


### Let's define the network
We need to normalize the dataset since it is quantized to 4 bits
(So let's devide the input by 15)

In [ ]:
# Define a simple model using NeuronovaSim
class SimpleSNN(torch.nn.Module):
    def __init__(
        self,
        input_size=16,
        hidden_size=24,
        output_size=4,
        norm_factor=15,  # We need to normalize the dataset with 4bit integer precision
        dt=1e-3,
        add_mismatch=False,
        bits_synapse=None,  # Set to a number to enable quantization
    ):
        super().__init__()

        self.norm_factor = torch.tensor(norm_factor)

        # Define network architecture - neurons per layer
        neurons_per_layer = [hidden_size, hidden_size, hidden_size, output_size]

        # Create the NeuronovaSim instance
        # Parameters: network type, model type, input size, neurons per layer
        self.sim = NeuronovaSim(
            chip_topoplogy == ["FF", "FF"],
            chip_model == "HWModel_1",
            input_size=input_size,
            neuron_number_per_layer=neurons_per_layer,
        )

        print(
            f"Created NeuronovaSim with {input_size} inputs and {len(neurons_per_layer)} layers:"
        )
        print(f"  - Layer 1: {neurons_per_layer[0]} neurons")
        print(f"  - Layer 2: {neurons_per_layer[1]} neurons")
        print(f"  - Layer 3: {neurons_per_layer[2]} neurons")
        print(f"  - Output: {neurons_per_layer[3]} neurons")

    def forward(self, x):
        # x is expected to be (batch_size, time_steps, input_features)
        device = x.device

        # Normalize the input
        # This is used to normalize the first layer between 0 and 1
        # based on the maximum firing rate from the dataset
        x_normalized = x / self.norm_factor

        # Call the simulator, which returns:
        # (cur1, cur2, cur3, cur4, spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4)
        outs = self.sim.forward(x_normalized)

        # Unpack the simulator outputs
        (
            spk1,
            spk2,
            spk3,
            spk4,
            mem1,
            mem2,
            mem3,
            mem4,
            cur1,
            cur2,
            cur3,
            cur4,
        ) = outs

        # Rearrange the outputs from (batch, time, features) to (time, batch, features)
        #
        cur1 = cur1.permute(1, 0, 2)
        cur2 = cur2.permute(1, 0, 2)
        cur3 = cur3.permute(1, 0, 2)
        cur4 = cur4.permute(1, 0, 2)

        spk1 = spk1.permute(1, 0, 2)
        spk2 = spk2.permute(1, 0, 2)
        spk3 = spk3.permute(1, 0, 2)
        spk4 = spk4.permute(1, 0, 2)

        mem1 = mem1.permute(1, 0, 2)
        mem2 = mem2.permute(1, 0, 2)
        mem3 = mem3.permute(1, 0, 2)
        mem4 = mem4.permute(1, 0, 2)

        # Return all outputs for detailed analysis
        return (spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4, cur1, cur2, cur3, cur4)


# Parameters
input_size = 16
hidden_size = 24
output_size = 4
norm_factor = 15  # Used to normalize input between 0 and 1


# Create the SNN model
model = SimpleSNN(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    norm_factor=norm_factor,
    dt=1e-3,
    add_mismatch=False,
    bits_synapse=None,  # Set to a number to enable quantization
)


# weight_range = (0.6, 2)
# model.sim.get_layers()[0].init_weight(weight_range)
# model.sim.get_layers()[2].init_weight(weight_range)
# model.sim.get_layers()[4].init_weight(weight_range)
# model.sim.get_layers()[6].init_weight(weight_range)

# model.sim.get_layers()[0].fake_quant.set_w_range(w_min=1.3, w_max=2)
# model.sim.get_layers()[2].fake_quant.set_w_range(w_min=1.3, w_max=2)
# model.sim.get_layers()[4].fake_quant.set_w_range(w_min=1.3, w_max=2)
# model.sim.get_layers()[6].fake_quant.set_w_range(w_min=1.3, w_max=2)

# Print model information
print(
    f"\nModel created with input_size={input_size}, hidden_size={hidden_size}, output_size={output_size}"
)

TypeError: NeuronovaSim.__init__() got an unexpected keyword argument 'network_type'

### Define Loss Function

In [ ]:
# Helper function to compute accuracy
def compute_loss_and_metrics(
    spk_rec,
    labels,
):
    ### Regularizer Params ###
    rate_offset = 0.25
    cross_reg_weight = 1000

    ### Torch Loss function
    loss_function = nn.CrossEntropyLoss()

    activity = spk_rec.sum(dim=1)  # -> (B, C)

    # extract class-indices
    _, targets = labels.max(-1)  # -> (B,)

    # cross-entropy on (B,C) vs (B,)
    main_loss = loss_function(activity, targets)

    # firing-rate regularization
    time_steps = spk_rec.size(1)  # ⟵ use time-axis
    avg_firing_rate = activity.mean() / time_steps
    rate_reg = (avg_firing_rate - rate_offset).pow(2).mean()

    loss = main_loss + cross_reg_weight * rate_reg

    # now compute accuracy on same summed output
    final_out = spk_rec.sum(dim=1)

    _, predicted = final_out.max(-1)  # -> (B,)
    _, targets = labels.max(-1)  # -> (B,)

    correct = predicted.eq(targets).sum().item()
    total = targets.numel()
    accuracy = correct / total if total > 0 else 0.0

    return loss, accuracy, predicted, targets

### Train and plot function

In [ ]:
# Helper function to visualize SNN activity
def plot_snn_activity(spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4, sample_idx=0):
    """
    Plots the membrane potential and spikes for each layer.
    """
    fig, axs = plt.subplots(4, 2, figsize=(15, 10))
    fig.suptitle(f"SNN Activity for Sample {sample_idx}", fontsize=14)

    layer_names = ["Layer 1", "Layer 2", "Layer 3", "Output Layer"]
    mem_arrs = [mem1, mem2, mem3, mem4]
    spk_arrs = [spk1, spk2, spk3, spk4]

    for i in range(4):
        if mem_arrs[i] is not None:
            # Select one neuron from each layer for visualization
            axs[i, 0].plot(mem_arrs[i][sample_idx, :, :].cpu().detach().numpy())
            axs[i, 0].set_title(f"{layer_names[i]} - Membrane Potentials")
            axs[i, 0].set_xlabel("Time Step")
            axs[i, 0].set_ylabel("Vmem")

        if spk_arrs[i] is not None:
            # Plot spike raster for a few neurons
            spike_data = spk_arrs[i][sample_idx, :, :].cpu().detach().numpy()
            for n in range(min(10, spike_data.shape[1])):  # Plot up to 10 neurons
                spike_times = np.where(spike_data[:, n] > 0.5)[0]
                axs[i, 1].scatter(
                    spike_times, [n] * len(spike_times), marker="|", color="black", s=20
                )

            axs[i, 1].set_title(f"{layer_names[i]} - Spike Raster")
            axs[i, 1].set_xlabel("Time Step")
            axs[i, 1].set_ylabel("Neuron Index")
            axs[i, 1].set_ylim(-0.5, min(10, spike_data.shape[1]) - 0.5)

    plt.tight_layout()
    plt.show()


# Function to train the SNN for a few epochs
def train_snn(model, dataloader, num_epochs=5, loss_offset=0.5):
    """
    Training loop for the SNN using concepts from the original experiment code.
    """
    # Define optimizer (similar to the original code which used Adamax)
    optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)

    # Training statistics tracking (similar to mean_fires in the original)
    layer_stats = {
        "Firing activity layer 1": 0,
        "Firing activity layer 2": 0,
        "Firing activity layer 3": 0,
        "Firing activity layer 4": 0,
        "Train Batches": 0,
    }

    # Training loop
    for epoch in range(num_epochs):
        # Reset epoch statistics
        total_loss = 0
        total_acc = 0
        num_batches = 0

        # Reset layer statistics
        for key in layer_stats:
            layer_stats[key] = 0

        for batch_idx, (inputs, labels, _) in enumerate(dataloader):
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Unpack outputs
            spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4, cur1, cur2, cur3, cur4 = (
                outputs
            )

            # Calculate accuracy and loss with the updated function
            loss, accuracy, predicted, targets = compute_loss_and_metrics(spk4, labels)

            # Backward pass and optimization
            loss.backward()

            # Add gradient clipping like in the original
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Track firing activities (similar to the original code)
            with torch.no_grad():
                layer_stats["Firing activity layer 1"] += spk1.mean().item()
                layer_stats["Firing activity layer 2"] += spk2.mean().item()
                layer_stats["Firing activity layer 3"] += spk3.mean().item()
                layer_stats["Firing activity layer 4"] += spk4.mean().item()
                layer_stats["Train Batches"] += 1

            total_loss += loss.item()
            total_acc += accuracy
            num_batches += 1

            if batch_idx == 0:

                if epoch % 25 == 0:
                    # Visualize network activity for the first sample in the batch
                    plot_snn_activity(spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4)

        # Epoch summary
        avg_loss = total_loss / num_batches
        avg_acc = total_acc / num_batches

        # Calculate average firing rates
        avg_firing_1 = (
            layer_stats["Firing activity layer 1"] / layer_stats["Train Batches"]
        )
        avg_firing_2 = (
            layer_stats["Firing activity layer 2"] / layer_stats["Train Batches"]
        )
        avg_firing_3 = (
            layer_stats["Firing activity layer 3"] / layer_stats["Train Batches"]
        )
        avg_firing_4 = (
            layer_stats["Firing activity layer 4"] / layer_stats["Train Batches"]
        )

        print(f"Epoch {epoch+1} completed:")
        print(f"  - Avg Loss: {avg_loss:.4f}")
        print(f"  - Avg Accuracy: {avg_acc:.4f}")
        print(
            f"  - Avg Firing Rates: L1={avg_firing_1:.4f}, L2={avg_firing_2:.4f}, L3={avg_firing_3:.4f}, L4={avg_firing_4:.4f}"
        )

        # Check if firing rates are too low (early stopping like in original)
        if avg_firing_4 <= 0.001:
            print(
                "Warning: Very low firing rate in output layer. Training may be unstable."
            )

    return model

### Training and validation 

In [1]:
# Grab one batch to visualize initial network activity

inputs, labels, _ = next(iter(train_loader))

print(f"\nInput shape: {inputs.shape}")
print(f"Label shape: {labels.shape}")

# Do a forward pass
with torch.no_grad():
    print("\nRunning initial forward pass...")
    outputs = model(inputs)
    spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4, cur1, cur2, cur3, cur4 = outputs

    # Print output shapes
    print(f"Output spike shape (layer 4): {spk4.shape}")
    print(f"Output membrane shape (layer 4): {mem4.shape}")

    # Calculate initial accuracy
    loss, accuracy, predicted, targets = compute_loss_and_metrics(spk4, labels)
    print(f"Initial accuracy: {accuracy:.4f}")

    # Visualize network activity
    plot_snn_activity(spk1, spk2, spk3, spk4, mem1, mem2, mem3, mem4)

# Train the model
print("\nTraining the SNN model...")
trained_model = train_snn(model, train_loader, num_epochs=100)

print("\nTraining completed!")

# Test the trained model on validation set
print("\nEvaluating the trained model on validation set...")

with torch.no_grad():
    total_acc = 0
    num_batches = 0

    for inputs, labels, _ in val_loader:
        outputs = trained_model(inputs)
        spk4 = outputs[3]  # Output layer spikes

        loss, accuracy, predicted, targets = compute_loss_and_metrics(spk4, labels)
        total_acc += accuracy
        num_batches += 1

    avg_acc = total_acc / num_batches
    print(f"Validation accuracy: {avg_acc:.4f}")


print("\nTutorial completed!")

NameError: name 'train_loader' is not defined